<a href="https://colab.research.google.com/github/sohamparikh94/if-hw2/blob/master/my_game_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install neuralcoref
!pip install spacy==2.1.3
!pip install pymagnitude
!pip install git+https://github.com/huggingface/transformers.git@master#egg=transformers
!pip install allennlp
!wget -nc http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude
!python -m spacy download en
!git clone https://github.com/sohamparikh94/if-hw2.git

In [0]:
from IPython import embed

import random
import spacy
import scipy
import neuralcoref
from pymagnitude import *
from allennlp.predictors.predictor import Predictor
from transformers import GPT2LMHeadModel, GPT2Tokenizer

100%|██████████| 40155833/40155833 [00:05<00:00, 7697681.47B/s] 


In [0]:
class Game:
  """The Game class represents the world.  Internally, we use a 
     graph of Location objects and Item objects, which can be at a 
     Location or in the player's inventory.  Each locations has a set of
     exits which are the directions that a player can move to get to an
     adjacent location. The player can move from one location to another
     location by typing a command like "Go North".
  """

  def __init__(self, start_at):
    # start_at is the location in the game where the player starts
    self.curr_location = start_at
    self.curr_location.has_been_visited = True
    # inventory is the set of objects that the player has collected/
    self.inventory = {}
    # Print the special commands associated with items in the game (helpful 
    # for debugging and for novice players).
    self.print_commands = True
    self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    self.model = GPT2LMHeadModel.from_pretrained('gpt2')
    self.model.to('cuda')
    self.nlp = spacy.load('en')
    lines = list()
    self.documents = list()
    with open('if-hw2/paragraphs.txt') as f:
      for line in f:
        if(line.strip()):
          self.documents.append(line)

  def describe(self):
    """Describe the current game state by first describing the current 
       location, then listing any exits, and then describing any objects
       in the current location."""
    self.describe_current_location()
    self.describe_exits()
    self.describe_items()

  def describe_current_location(self):
    """Describe the current location by printing its description field."""
    print(self.curr_location.description)

  def describe_exits(self):
    """List the directions that the player can take to exit from the current
       location."""
    exits = []
    for exit in self.curr_location.connections.keys():
      exits.append(exit.capitalize())
    if len(exits) > 0:
      print("Exits: ", end = '')
      print(*exits, sep = ", ",)
  
  def describe_items(self):
    """Describe what objects are in the current location."""
    if len(self.curr_location.items) > 0:
      print("You see: ")
      for item_name in self.curr_location.items:
        item = self.curr_location.items[item_name]
        print(item.description)
        if self.print_commands:
          special_commands = item.get_commands()
          for cmd in special_commands:
            print('\t', cmd)

  def add_to_inventory(self, item):
    """Add an item to the player's inventory."""
    self.inventory[item.name] = item
  
  def is_in_inventory(self,item):
    return item.name in self.inventory

  def get_items_in_scope(self):
    """Returns a list of items in the current location and in the inventory"""
    items_in_scope = []
    for item_name in self.curr_location.items:
      items_in_scope.append(self.curr_location.items[item_name])
    for item_name in self.inventory:
      items_in_scope.append(self.inventory[item_name])
    return items_in_scope

In [0]:
class Location:
  """Locations are the places in the game that a player can visit.
     Internally they are represented nodes in a graph.  Each location stores
     a description of the location, any items in the location, its connections
     to adjacent locations, and any blocks that prevent movement to an adjacent
     location.  The connections is a dictionary whose keys are directions and
     whose values are the location that is the result of traveling in that 
     direction.  The travel_descriptions also has directions as keys, and its 
     values are an optional short desciption of traveling to that location.
  """
  def __init__(self, name, description, end_game=False):
    # A short name for the location
    self.name = name
    # A description of the location
    self.description = description
    # True if entering this location should end the game
    self.end_game = end_game
    # Dictionary mapping from directions to other Location objects
    self.connections = {}
    # Dictionary mapping from directions to text description of the path there
    self.travel_descriptions = {}
    # Dictionary mapping from item name to Item objects present in this location
    self.items = {}
    # Dictionary mapping from direction to Block object in that direction
    self.blocks = {}
    # Flag that gets set to True once this location has been visited by player
    self.has_been_visited = False

  def add_connection(self, direction, connected_location, travel_description=""):
    """Add a connection from the current location to a connected location.
       Direction is a string that the player can use to get to the connected
       location.  If the direction is a cardinal direction, then we also 
       automatically make a connection in the reverse direction."""
    self.connections[direction] = connected_location
    self.travel_descriptions[direction] = travel_description
    if direction == 'north':
      connected_location.connections["south"] = self
      connected_location.travel_descriptions["south"] = ""
    if direction == 'south':
      connected_location.connections["north"] = self
      connected_location.travel_descriptions["north"] = ""
    if direction == 'east':
      connected_location.connections["west"] = self
      connected_location.travel_descriptions["west"] = ""
    if direction == 'west':
      connected_location.connections["east"] = self
      connected_location.travel_descriptions["east"] = ""
    if direction == 'up':
      connected_location.connections["down"] = self
      connected_location.travel_descriptions["down"] = ""
    if direction == 'down':
      connected_location.connections["up"] = self
      connected_location.travel_descriptions["up"] = ""
    if direction == 'in':
      connected_location.connections["out"] = self
      connected_location.travel_descriptions["out"] = ""
    if direction == 'out':
      connected_location.connections["in"] = self
      connected_location.travel_descriptions["in"] = ""


  def add_item(self, name, item):
    """Put an item in this location."""
    self.items[name] = item

  def remove_item(self, item):
    """Remove an item from this location (for instance, if the player picks it
       up and puts it in their inventory)."""
    self.items.pop(item.name)


  def is_blocked(self, direction, game):
    """Check to if there is an obstacle in this direction."""
    if not direction in self.blocks:
        return False
    (block_description, preconditions) = self.blocks[direction]
    if check_preconditions(preconditions, game):
      # All the preconditions have been met.  You may pass.
      return False
    else: 
      # There are still obstalces to overcome or puzzles to solve.
      return True

  def get_block_description(self, direction):
    """Check to if there is an obstacle in this direction."""
    if not direction in self.blocks:
      return ""
    else:
      (block_description, preconditions) = self.blocks[direction]
      return block_description

  def add_block(self, blocked_direction, block_description, preconditions):
    """Create an obstacle that prevents a player from moving in the blocked 
       location until the preconditions are all met."""
    self.blocks[blocked_direction] = (block_description, preconditions)

In [0]:
def check_preconditions(preconditions, game, print_failure_reasons=True):
  """Checks whether the player has met all of the specified preconditions"""
  all_conditions_met = True
  for check in preconditions: 
    if check == "inventory_contains":
      item = preconditions[check]
      if not game.is_in_inventory(item):
        all_conditions_met = False
        if print_failure_reasons:
          print("You don't have the %s" % item.name)
    if check == "in_location":
      location = preconditions[check]
      if not game.curr_location == location:
        all_conditions_met = False
        if print_failure_reasons:
          print("You aren't in the correct location")
    if check == "location_has_item":
      item = preconditions[check]
      if not item.name in game.curr_location.items:
        all_conditions_met = False
        if print_failure_reasons:
          print("The %s isn't in this location" % item.name)
    if check == "is_ripe":
      item = preconditions[check]
      if(item.is_withered):
        print("%s is not ripe!" % item.name)
        all_conditions_met = False
    if check == 'is_full':
      item = preconditions[check]
      if(not item.is_full):
        print("%s is not full" % item.name)
        all_conditions_met = False
    if check == 'is_lit':
      item = preconditions[check]
      if(not item.is_lit):
        all_conditions_met = False
    if check == 'is_unlocked':
      item = preconditions[check]
      if(not item.is_unlocked):
        all_conditions_met = False
    # todo - add other types of preconditions
  return all_conditions_met


In [0]:
class Item:
  """Items are objects that a player can get, or scenery that a player can
     examine."""
  def __init__(self,
               name,
               description,
               examine_text="",
               take_text="",
               start_at=None,
               gettable=True,
               end_game=False):
    # The name of the object
    self.name = name
    # The default description of the object.
    self.description = description
    # The detailed description of the player examines the object.
    self.examine_text = examine_text
    # Text that displays when player takes an object.
    self.take_text = take_text if take_text else ("You take the %s." % self.name)
    # Indicates whether a player can get the object and put it in their inventory.
    self.gettable = gettable
    # True if entering this location should end the game.
    self.end_game = end_game
    # The location in the Game where the object starts.
    if start_at:
      start_at.add_item(name, self)
    self.commands = {}


  def get_commands(self):
    """Returns a list of special commands associated with this object"""
    return self.commands.keys()

  def add_action(self, command_text, function, arguments, preconditions={}):
    """Add a special action associated with this item"""
    self.commands[command_text] = (function, arguments, preconditions)

  def do_action(self, command_text, game):
    """Perform a special action associated with this item"""
    end_game = False  # Switches to True if this action ends the game.
    if command_text in self.commands:
      function, arguments, preconditions = self.commands[command_text]
      if check_preconditions(preconditions, game):
        end_game = function(game, arguments)
    else:
      print("Cannot perform the action %s" % command_text)
    return end_game

In [0]:
class Parser:
  """The Parser is the class that handles the player's input.  The player 
     writes commands, and the parser performs natural language understanding
     in order to interpret what the player intended, and how that intent
     is reflected in the simulated world. 
  """
  def __init__(self, game, item_list, location_list):
    # A list of all of the commands that the player has issued.
    self.command_history = []
    # A pointer to the game.
    self.game = game
    self.generate_known_commands(item_list, location_list)
    self.vectors = Magnitude('glove.6B.300d.magnitude')
    self.dep_parse = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/biaffine-dependency-parser-ptb-2018.08.23.tar.gz")
    self.coref = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/coref-model-2018.02.05.tar.gz")
    self.nlp = spacy.load('en')
    neuralcoref.add_to_pipe(self.nlp)


  def generate_known_commands(self, item_list, location_list):

    self.known_commands = ['look', 'inventory', 'l', 'i', 'jump', 'n', 's', 'e', 'w']
    predefined_commands = ['take', 'get', 'go', 'examine', 'drop', 'jump', 'look']
    directions = ['north', 'south', 'east', 'west', 'up', 'down', 'out', 'in']
    for item in item_list:
      for command in predefined_commands:
        self.known_commands.append(' '.join([command, item.name]))
      for command in item.commands:
        self.known_commands.append(command)
    for direction in directions:
      self.known_commands.append(direction)
      for command in predefined_commands:
        self.known_commands.append(' '.join([command, item.name]))



  def coref_resolution(self, command):
    try:
      coref = self.coref_predictor.predict(command)
      for cluster in cor['clusters']:
        first_occurrence = cluster[0]
        actual_name = ' '.join(new_words[first_occurrence[0]:first_occurrence[-1]+1])
        for other_occurrences in cluster[1:]:
          new_words[other_occurrences[0]] = actual_name
          for idx in other_occurrences[1:]:
            if(idx > other_occurrences[0]):
              new_words[idx] = ''
      new_command = ' '.join(new_words)
    except:
      doc = self.nlp(command)
      new_tokens = [token for token in doc]
      for cluster in doc._.coref_clusters:
        for mention in cluster.mentions[1:]:
          new_tokens[mention.start] = cluster.main
          for idx in range(mention.start + 1, mention.end):
            new_tokens[idx] = None
      new_tokens = [token.text for token in new_tokens if(token)]
      new_command = ' '.join(new_tokens)

    return new_command


  def get_conjunctions(self, commands):
    split_commands = commands.split(',')
    all_commands = list()
    for command in split_commands:
        doc = self.nlp(command)
        curr_command = list()
        for token in doc:
            if(token.dep_ == 'cc'):
                all_commands.append(' '.join(curr_command).strip())
                curr_command = list()
            elif(token.dep_ != 'then'):
                curr_command.append(token.text)
        all_commands.append(' '.join(curr_command).strip())
    return all_commands



  def construct_sentence_vector(self, command):
    sentence_vector = np.zeros(shape=(self.vectors.dim,))
    for word in command.split():
      word_vector = self.vectors.query(word)
      sentence_vector += word_vector
    sentence_vector = sentence_vector/len(command.split())

    return sentence_vector


  def find_most_similar_command(self, user_command):
    user_vector = self.construct_sentence_vector(user_command)
    similarities = list()
    for command in self.known_commands:
      command_vector = self.construct_sentence_vector(command)
      similarities.append(1 - scipy.spatial.distance.cosine(user_vector, command_vector))

    return self.known_commands[np.argmax(similarities)]


  def get_player_intent(self,command):
    command = command.lower()

    if "," in command:
      # Let the player type in a comma separted sequence of commands
      return "sequence"
    elif self.get_direction(command):
      # Check for the direction intent
      return "direction"
    elif command.lower() == "look" or command.lower() == "l":
      # when the user issues a "look" command, re-describe what they see
      return "redescribe"
    elif "examine " in command or command.lower().startswith("x "):
      return "examine"
    elif  "take " in command or "get " in command:
      return "take"
    elif "drop " in command:
      return "drop"
    elif "inventory" in command or command.lower() == "i":
      return "inventory"
    else: 
      for item in self.game.get_items_in_scope():
        special_commands = item.get_commands()
        for special_command in special_commands:
          if command == special_command.lower():
            return "special"


  def get_command_list(self, command):
    command = self.coref_resolution(command)
    command_list = self.get_conjunctions(command)

    return command_list


  def parse_command(self, command):
    # add this command to the history
    self.command_history.append(command)

    # By default, none of the intents end the game. The following are ways this
    # flag can be changed to True.
    # * Going to a certain place.
    # * Entering a certain special command
    # * Picking up a certain object.

    end_game = False

    # Intents are functions that can be executed

    # intent = self.get_player_intent(command)
    command_list = self.get_command_list(command)
    command = ','.join(command_list)
    if(',' not in command):
      command = self.find_most_similar_command(command)
    intent = self.get_player_intent(command)
    if intent == "direction":
      end_game = self.go_in_direction(command)
    elif intent == "redescribe":
      self.game.describe()
    elif intent == "examine":
      self.examine(command)
    elif intent == "take":
      end_game = self.take(command)
    elif intent == "drop":
      self.drop(command)
    elif intent == "inventory":
      self.check_inventory(command)
    elif intent == "special":
      end_game = self.run_special_command(command)
    elif intent == "sequence":
      end_game = self.execute_sequence(command)
    else:
      print("I'm not sure what you want to do.")
    return end_game

  ### Intent Functions ###

  def go_in_direction(self, command):
    """ The user wants to in some direction """
    direction = self.get_direction(command)

    if direction:
      if direction in self.game.curr_location.connections:
        if self.game.curr_location.is_blocked(direction, self.game):
          # check to see whether that direction is blocked.
          print(self.game.curr_location.get_block_description(direction))
        else:
          # if it's not blocked, then move there 
          self.game.curr_location = self.game.curr_location.connections[direction]
          if(self.game.curr_location.name == "Basement" and not self.game.has_magic):
            print("The gas has suffocated you to death")
            return True

          # If moving to this location ends the game, only describe the location
          # and not the available items or actions.
          if self.game.curr_location.end_game:
            self.game.describe_current_location()
            if(check_for_win(self.game)):
              print("You could escape successfully")
            else:
              print("You could not escape successfully because the guards have recognized you")
          else:
            self.game.describe()
      else:
        print("You can't go %s from here." % direction.capitalize())
    return self.game.curr_location.end_game

  def check_inventory(self,command):
    """ The player wants to check their inventory"""
    if len(self.game.inventory) == 0:
      print("You don't have anything.")
    else:
      descriptions = []
      for item_name in self.game.inventory:
        item = self.game.inventory[item_name]
        descriptions.append(item.description)
      print("You have: ", end = '')
      print(*descriptions, sep = ", ",)
  

  def examine(self, command):
    """ The player wants to examine something """
    command = command.lower()
    matched_item = False
    # check whether any of the items at this location match the command
    for item_name in self.game.curr_location.items:
      if item_name in command:
        item = self.game.curr_location.items[item_name]
        if item.examine_text:
          print(item.examine_text)
          matched_item = True
        break
    # check whether any of the items in the inventory match the command
    for item_name in self.game.inventory:
      if item_name in command:
        item = self.game.inventory[item_name]
        if item.examine_text:
          print(item.examine_text)
          matched_item = True
    # fail
    if not matched_item:
      print("You don't see anything special.")


  def take(self, command):
    """ The player wants to put something in their inventory """
    command = command.lower()
    matched_item = False

    # This gets set to True if posession of this object ends the game.
    end_game = False

    # check whether any of the items at this location match the command
    for item_name in self.game.curr_location.items:
      if item_name in command:
        item = self.game.curr_location.items[item_name]
        if item.gettable:
          self.game.add_to_inventory(item)
          self.game.curr_location.remove_item(item)
          print(item.take_text)
          end_game = item.end_game
        else:
          print("You cannot take the %s." % item_name)
        matched_item = True
        break
    # check whether any of the items in the inventory match the command
    if not matched_item:
      for item_name in self.game.inventory:
        if item_name in command:
          print("You already have the %s." % item_name)
          matched_item = True
    # fail
    if not matched_item:
      print("You can't find it.")

    return end_game

  def drop(self, command):
    """ The player wants to remove something from their inventory """
    command = command.lower()
    matched_item = False
    # check whether any of the items in the inventory match the command
    if not matched_item:
      for item_name in self.game.inventory:
        if item_name in command:
          matched_item = True
          item = self.game.inventory[item_name]
          self.game.curr_location.add_item(item_name, item)
          self.game.inventory.pop(item_name)
          print("You drop the %s." % item_name)
          break
    # fail
    if not matched_item:
      print("You don't have that.")


  def run_special_command(self, command):
    """Run a special command associated with one of the items in this location
       or in the player's inventory"""
    for item in self.game.get_items_in_scope():
        special_commands = item.get_commands()
        for special_command in special_commands:
          if command == special_command.lower():
            return item.do_action(special_command, self.game)

  def execute_sequence(self, command):
    for cmd in command.split(","):
      cmd = cmd.strip()
      self.parse_command(cmd)

  def get_direction(self, command):
    command = command.lower()
    if command == "n" or "north" in command:
      return "north" 
    if command == "s" or "south" in command:
      return "south"
    if command == "e" or "east" in command: 
      return "east"
    if command == "w" or "west" in command:
      return "west"
    if command == "up":
      return "up"
    if command == "down":
      return "down"
    if command.startswith("go out"):
      return "out"
    if command.startswith("go in"):
      return "in"
    for exit in self.game.curr_location.connections.keys():
      if command == exit.lower() or command == "go " + exit.lower():
        return exit
    return None

In [0]:
def add_item_to_inventory(game, *args):
  """ Add a newly created Item and add it to your inventory."""
  (item, action_description, already_done_description) = args[0]
  if(not game.is_in_inventory(item)):
    print(action_description)
    game.add_to_inventory(item)
  else:
    print(already_done_description)
  return False

def describe_something(game, *args):
  """Describe some aspect of the Item"""
  (description) = args[0]
  print(description)
  return False

def destroy_item(game, *args):
  """Removes an Item from the game by setting its location is set to None."""
  (item, action_description) = args[0]
  if game.is_in_inventory(item):
    game.inventory.pop(item.name)
    print(action_description)
  elif item.name in game.curr_location.items:
    game.curr_location.remove_item(item)
    print(action_description)
  else:
    print(already_done_description)
  return False

def end_game(game, *args):
  """Ends the game."""
  end_message = args[0]
  print(end_message)
  return True

# def pick_item(game, *args):

#   (item, precondition_item) = args[0]
#   can_pick = True  
#   if(precondition_item.name not in game.inventory):
#     can_pick = False
#   if(can_pick):
#     add_item_to_inventory(game, (item, "%s have been in put in %s" % (item.name, precondition_item.name), "%s is already inside the %s" % (item.name, precondition_item.name)))
#   else:
#     print("You can't just pick them up like that!")
  return False

def fill_water(game, *args):
  (vessel, description, already_done_description) = args[0]
  if(vessel.is_full):
    print(already_done_description)
  else:
    vessel.is_full = True
    print(description)

  return False


def water_plant(game, *args):
  (plant, jug, description, already_done_description) = args[0]
  if(plant.is_withered):
    plant.is_withered = False
    jug.is_full = False
    print(description)
  else:
    print(already_done_description)

  return False

def wear_item(game, *args):

  (item, description, already_done_description) = args[0]
  if(not item.is_wearing):
    item.is_wearing = True
    print(description)
  else:
    print(already_done_description)

  return False


def light_item(game, *args):

  (item, description, already_done_description) = args[0]
  if(not item.is_lit):
    item.is_lit = True
    print(description)
  else:
    print(already_done_description)


def get_magic(game, *args):

  (plant, description) = args[0]
  destroy_item(game, (plant, description))
  game.has_magic = True

  return False


def break_glass(game, *args):

  (case, armor_set, description) = args[0]
  destroy_item(game, (case, description))
  game.curr_location.add_item(armor_set.name, armor_set)

  return False

def check_for_win(game):
  win = True
  if('armor' in game.inventory):
    if(not game.inventory['armor'].is_wearing):
      win = False
  else:
    win = False
  if('spear' not in game.inventory):
    win = False

  return win

def unlock(game, *args):

  door = args[0]
  current_doc = random.choice(game.documents)
  doc = game.nlp(current_doc)
  sents = [sent.text for sent in list(doc.sents)]
  starting = ' '.join(sents[:2])
  encoded_inputs = game.tokenizer.encode(starting, add_special_tokens=False, return_tensors='pt')
  encoded_inputs = encoded_inputs.to('cuda')
  print("Generating Text")
  output_sequence = game.model.generate(input_ids=encoded_inputs, max_length=60, temperature=1.0, top_k=0, top_p=0.9, repetition_penalty=1.0)
  print("Sequence Generated\n\n")
  generated_sequence = output_sequence[0].tolist()
  text = game.tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
  # text = ' '.join(' '.join([sent.text for sent in list(game.nlp(text).sents)]))
  text = ' '.join(text.split('\n'))

  human_text = starting[:]
  for sent in sents[2:]:
    if(abs(len(human_text) - len(text)) < 20):
      break
    human_text += ' ' + sent

  if(random.random() < 0.5):
    print(text)
    print("+++++++++++")
    print(human_text)
    prompt_text = input("You are given two pieces of texts which start with the same pair of sentences. One of them is generated by AI while the other is written by the king. A true guard can always detect the text generated by the king. Press 1 if you think the first is generated by our king. Press 2 otherwise.\n")
    if(prompt_text.strip() == '2'):
      door.is_unlocked = True
      print("Hurray! You are a true guard! The door is now unlocked.")
    else:
      print("WRONG ANSWER! Are you even a guard?")
  else:
    print(human_text)
    print("+++++++++++")
    print(text)
    prompt_text = input("\n\nYou are given two pieces of texts which start with the same pair of sentences. One of them is generated by AI while the other is written by the king. A true guard can always detect the text generated by the king. Press 1 if you think the first is generated by our king. Press 2 otherwise.\n\n")
    if(prompt_text.strip() == '1'):
      door.is_unlocked = True
      print("Hurray! You are a true guard! The door is now unlocked.")
    else:
      print("WRONG ANSWER! Are you even a guard?")

  return False

In [0]:
def build_game():
  # Locations
  bedroom = Location("Bedroom", "This is your bedroom.")
  kitchen = Location("Kitchen", "This is your kitchen. There is a sink over here.")
  hall1 = Location("Hall 1", "This is Hall 1.")
  attic = Location("Attic", "This is the attic. It is in complete darkness.")
  hall2 = Location("Hall 2", "This is Hall 2.")
  garden = Location("Garden", "This is the garden.")
  basement_stairs = Location("Basement Stairs", "You are on the basement stairs. These lead to the basement. There is a pungent smell of a deadly, toxic gas from the basement")
  basement = Location("Basement", "This is the basement. It is filled with a gas that will suffocate you to death if you move further!")
  armor_room = Location("Armor Room", "This is ths armor room")
  attic_stairs = Location("Attic Stairs", "You are on the attic stairs. These stairs lead to the attic, which is in complete darkness")
  outside = Location("Outside", "You are outside the house now", end_game=True)
  # heater_room = Location("Heater Room", "This is the heater room.")
  
  # cottage = Location("Cottage", "You are standing in a small cottage.")
  # garden_path = Location("Garden Path", "You are standing on a lush garden path. There is a cottage here.")
  # cliff = Location("Cliff", "There is a steep cliff here. You fall off the cliff and lose the game. THE END.", end_game=True)
  # fishing_pond = Location("Fishing Pond", "You are at the edge of a small fishing pond.")
  # # Connections
  hall1.add_connection("down", basement_stairs)
  hall1.add_connection("south", hall2)
  hall1.add_connection("east", kitchen)
  hall1.add_connection("west", outside)
  hall2.add_connection("east", bedroom)
  hall2.add_connection("up", attic_stairs)
  attic_stairs.add_connection("up", attic)
  hall2.add_connection("south", garden)
  basement_stairs.add_connection("down", basement)
  basement.add_connection("south", armor_room)
  # cottage.add_connection("out", garden_path)
  # garden_path.add_connection("west", cliff)
  # garden_path.add_connection("south", fishing_pond)

  # Items that you can pick up
  jug = Item("jug", "a water jug", start_at=bedroom)
  torch = Item("torch", "a torch", start_at=basement)
  plant = Item("plant", "a magical plant", start_at=garden, gettable=False)
  spear = Item("spear", "a spear", start_at=attic)
  armor_set = Item("armor", "a complete armor set", start_at=None)
  glass_case = Item("glass case", "a glass case with armor inside", start_at=armor_room, gettable=False)
  out_door = Item("door", "this door leads to the outside. To unlock it, you need to be good at detecting AI generated text.", start_at=hall1, gettable=False)
  




  # fishing_pole = Item("pole", "a fishing pole", "A SIMPLE FISHING POLE.", start_at=cottage)
  # potion = Item("potion", "a poisonous potion", "IT'S BRIGHT GREEN AND STEAMING.", start_at=cottage, take_text='As you near the potion, the fumes cause you to faint and lose the game. THE END.', end_game=True)
  # rosebush = Item("rosebush", "a rosebush", "THE ROSEBUSH CONTAINS A SINGLE RED ROSE.  IT IS BEAUTIFUL.", start_at=garden_path)
  # rose = Item("rose", "a red rose", "IT SMELLS GOOD.",  start_at=None)
  # fish = Item("fish", "a dead fish", "IT SMELLS TERRIBLE.", start_at=None)

  # # Sceneary (not things that you can pick up)
  # pond = Item("pond", "a small fishing pond", "THERE ARE FISH IN THE POND.", start_at=fishing_pond, gettable=False)

  # Add special functions to your items
  jug.add_action("fill jug with water", fill_water, (jug, "The jug is now filled with water", "The jug is already filled with water"), preconditions={'in_location': kitchen, 'inventory_contains': jug})
  torch.add_action("light torch", light_item, (torch, "The torch is now lit", "The torch is already lit"), preconditions={'inventory_contains': torch})
  plant.add_action("water plant", water_plant, (plant, jug, "The plant is no longer withered", "The plant has already been watered!"), preconditions={'inventory_contains': jug, 'is_full': jug})
  plant.add_action("eat plant", get_magic, (plant, "By eating this magical plant, you have now gained powers to breathe in any gas without dying"), preconditions={'in_location': garden, 'is_ripe': plant})
  armor_set.add_action("wear armor", wear_item, (armor_set, "You have now worn the armor", "You already have the armor on!"), preconditions={'inventory_contains': armor_set})
  glass_case.add_action("break case", break_glass, (glass_case, armor_set, "The case has been broken"), preconditions={"inventory_contains": spear})
  out_door.add_action("unlock door", unlock, (out_door), preconditions={"inventory_contains": spear, "is_wearing": armor_set})

  # Blocked Actions

  attic_stairs.add_block("up", "You cannot enter the attic without any light", preconditions={'is_lit': torch, 'inventory_contains': torch})
  hall1.add_block("west", "This door needs to be unlocked in order to go outside.", preconditions={'is_unlocked': out_door})



  # rosebush.add_action("pick rose",  add_item_to_inventory, (rose, "You pick the lone rose from the rosebush.", "You already picked the rose."))
  # rose.add_action("smell rose",  describe_something, ("It smells sweet."))
  # pond.add_action("catch fish",  describe_something, ("You reach into the pond and try to catch a fish with your hands, but they are too fast."))
  # pond.add_action("catch fish with pole",  add_item_to_inventory, (fish, "You dip your hook into the pond and catch a fish.","You weren't able to catch another fish."), preconditions={"inventory_contains":fishing_pole})
  # fish.add_action("eat fish",  end_game, ("That's disgusting! It's raw! And definitely not sashimi-grade! But you've won this version of the game. THE END."))

  # return Game(cottage)
  plant.is_withered = True
  jug.is_full = False
  armor_set.is_wearing = False
  torch.is_lit = False
  out_door.is_unlocked = False

  game = Game(bedroom)
  game.has_magic = False

  location_list = [bedroom, kitchen, hall1, attic, hall2, garden, basement_stairs, basement, armor_room, attic_stairs, outside]
  item_list = [jug, torch, plant, spear, armor_set, glass_case, out_door]


  return game, item_list, location_list

In [0]:
def game_loop():
  game, item_list, location_list = build_game()
  parser = Parser(game, item_list, location_list)
  game.describe()

  command = ""
  while not (command.lower() == "exit" or command.lower == "q"):
    command = input(">")
    end_game = parser.parse_command(command)
    if end_game:
      return
print("You are inside a house from which you need to escape dressed as a guard. If you don't collect all the things that a guard wears (an armor set and a spear), there are guards outside who will catch you.")
game_loop()
print('THE GAME HAS ENDED.')

In [0]:
from google.colab import files

uploaded = files.upload()

Saving paragraphs.txt to paragraphs.txt


In [0]:
from graphviz import Digraph
from IPython.display import Image
import queue

def DFS(game, graph):
  """Do a depth-first-search traversal of the locations in the game
     starting at the start location, and create a GraphViz graph 
     to vizualize the connections between the locations, and the items
     that are located at each location."""
  start_location = game.curr_location
  frontier = queue.Queue()
  frontier.put(start_location)
  visited = {}
  visited[start_location.name] = True

  while not frontier.empty():
    current_location = frontier.get()
    game.curr_location = current_location
    name = current_location.name
    description = current_location.description
    items = current_location.items
    items_html = describe_items(current_location)
    html = "<<b>%s</b><br />%s<br />%s>" % (name, description, items_html)
    # Create a new node in the graph for this location
    graph.node(name, label=html)  

    connections = current_location.connections
    for direction in connections.keys():
      next_location = connections[direction]
      if not current_location.is_blocked(direction, game):
        # Create an edge between the current location and its successor
        graph.edge(name, next_location.name, label=direction.capitalize())
      else:
        # Create a dotted edge for connected locations that are blocked
        block_description = "%s\n%s" % (direction.capitalize(), current_location.get_block_description(direction))
        graph.edge(name, next_location.name, label=block_description, style="dotted")
      if not next_location.name in visited:
        visited[next_location.name] = True
        frontier.put(next_location)

def describe_items(location, print_commands=True):
    """Describe what objects are in the current location."""
    items_html = ""
    if len(location.items.keys()) > 0:
      items_html = "You see: "
    for item_name in location.items:
      item = location.items[item_name]
      items_html += item.description
      if print_commands:
        special_commands = item.get_commands()
        for cmd in special_commands:
          items_html += "<br/><i>%s</i>" % cmd
    return items_html

def save_to_drive(graph):
  from google.colab import drive
  drive.mount('/content/drive/')
  graph.render('/content/drive/My Drive/game-visualization', view=True)  

graph = Digraph(node_attr={'color': 'lightblue2', 'style': 'filled'})
game, item_list, locatio = build_game()
DFS(game, graph)
#save_to_drive(graph)
graph

AttributeError: 'tuple' object has no attribute 'curr_location'